## Creating a Logging Bucket (One time thing)

In [ ]:
import boto3
print("Version: ", boto3.__version__)
#list all buckets in s3 that the user can see
print("Buckets:", *boto3.client("s3").list_buckets()["Buckets"], sep="\n  ")
#list current user's region
print("Region: ", boto3.Session().region_name)

from s3_bucket_helper import *

%load_ext autoreload
%autoreload 2

In [ ]:
s3_resource = boto3.resource("s3")
s3_client = boto3.client("s3")

In [ ]:
# logging_bucket_name, logging_bucket_response = gen_bucket("logging-bucket", s3_resource)
# logging_bucket_response
## giving the logging bucket the required permissions via a policy

Get list of bucket dictionaries using a prefix and/or a date or date interval as a filter

In [ ]:
get_buckets_with_name_date("logging", [datetime.date(2023,2,12), "2023-2-16"])

Get list of bucket dictionaries using tags as filters (Not started yet)

In [ ]:
s3_client.get_bucket_tagging(Bucket = "ecm-simdata-attar")

In [ ]:
s3_resource.BucketPolicy

In [ ]:
help(s3_client.get_bucket_logging)